# Лабораторная работа №5 по предмету "Теория кодирования".
Выполнили студенты: Степанов Матвей и Овчинников Егор 6403-010302D



In [243]:
import numpy as np
import math
from itertools import combinations, product
import random
from operator import itemgetter

# 5.1 Написать функцию формирования порождающей матрицы кода Рида-Маллера (r,m) в каноническом виде для произвольно заданных r и m.

In [244]:
# Формируем бинарную матрицу с 2^cols строк и cols столбцами (все возможные двоичные векторы длины cols)
def generate_binary_matrix(cols):
    return list(product([0, 1], repeat=cols))

In [245]:
# Вычисляем значение функции f для множества I
def compute_f_value(vector, subset):
    return np.prod([(vector[idx] + 1) % 2 for idx in subset])

In [246]:
# Формируем вектор V_I на основе множества I
def get_vector_V(subset, num_cols):
    if len(subset) == 0:
        return np.ones(2 ** num_cols, int)
    else:
        v_vector = []
        for binary_vector in generate_binary_matrix(num_cols):
            f_value = compute_f_value(binary_vector, subset)
            v_vector.append(f_value)
        return v_vector

In [247]:
# Генерация всех возможных комбинаций I длиной не более r
def generate_combinations(num_cols, max_size):
    indices = range(num_cols)
    return [subset for subset_size in range(max_size + 1) for subset in combinations(indices, subset_size)]

In [248]:
# Вычисляем размер порождающей матрицы кода Рида-Маллера
def calculate_rm_matrix_size(r, m):
    return sum(math.comb(m, i) for i in range(r + 1))

In [249]:
# Формирование порождающей матрицы G для кода Рида-Маллера
def generate_rm_matrix(r, m):
    matrix_size = calculate_rm_matrix_size(r, m)
    matrix = np.zeros((matrix_size, 2 ** m), dtype=int)
    row_index = 0
    for subset in generate_combinations(m, r):
        matrix[row_index] = get_vector_V(subset, m)
        row_index += 1
    return matrix

In [250]:
# Генерация порождающей матрицы кода Рида-Маллера
G = generate_rm_matrix(2, 4)
print("Порождающая матрица:\n", G)

Порождающая матрица:
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]]


# 5.2. Реализовать алгоритм мажоритарного декодирования для кода Рида-Маллера.

In [251]:
# Сортировка для декодирования
def sort_for_major(m, r):
    indices = range(m)

    combinations_list = list(combinations(indices, r))
    if combinations_list:
        combinations_list.sort(key=lambda x: len(x))

    result = np.array(combinations_list, dtype=int)
    return result

In [252]:
def get_vector_H(I, m):
    return [word for word in generate_binary_matrix(m) if compute_f_value(word, I) == 1]

In [253]:
# Формирование комплиментарного множества
def get_Komplement(I, m):
    return [i for i in range(m) if i not in I]

In [254]:
def compute_f_t_value(words, I, t):
    return np.prod([(words[j] + t[j] + 1) % 2 for j in I])

In [255]:
def get_V_I_t(I, m, t):
    if not I:
        return np.ones(2 ** m, dtype=int)
    return [compute_f_t_value(word, I, t) for word in generate_binary_matrix(m)]

In [256]:
# Мажоритарное декодирование
def major_algorithm(w, r, m, size):
    i = r
    w_r = w.copy()
    Mi = np.zeros(size, dtype=int)
    max_weight = 2**(m - r - 1) - 1
    index = 0

    while True:
        for J in sort_for_major(m, i):
            max_zeros_and_ones_count = 2**(m - i - 1)
            zeros_count = 0
            ones_count = 0
            for t in get_vector_H(J, m):
                komplement = get_Komplement(J, m)
                V = get_V_I_t(komplement, m, t)
                c = np.dot(w_r, V) % 2

                if c == 0:
                    zeros_count += 1
                else:  # c == 1
                    ones_count += 1

            if zeros_count > max_weight and ones_count > max_weight:
                return
            if zeros_count > max_zeros_and_ones_count:
                Mi[index] = 0
                index += 1
            if ones_count > max_zeros_and_ones_count:
                Mi[index] = 1
                index += 1
                V = get_vector_V(J, m)
                w_r = (w_r + V) % 2

        if i > 0:
            if len(w_r) < max_weight:
                for J in sort_for_major(m, r + 1):
                    Mi[index] = 0
                    index += 1
                break
            i -= 1
        else:
            break

    reversed(Mi)
    return Mi

In [257]:
# Генерация слова с указанным количеством ошибок
def generate_word_with_n_mistakes(G, error_count):
    u = np.array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
    print("Исходное сообщение: ", u)

    u = u.dot(G) % 2
    mistake_pos = np.random.choice(len(u), size=error_count, replace=False)
    u[mistake_pos] = (u[mistake_pos] + 1) % 2

    return u

# 5.3. Провести экспериментальную проверку алгоритма декодирования для кода Рида-Маллера (2,4).

In [258]:
# Эксперимент для однократной ошибки
Err = generate_word_with_n_mistakes(G, 1)
print("Слово с однократной ошибкой:", Err)
print()

Decoded_word = major_algorithm(Err, 2, 4, len(G))
if Decoded_word is None:
    print("\nНеобходима повторная отправка сообщения")
else:
    print("Исправленное слово:", Decoded_word)
    V2 = Decoded_word.dot(G) % 2
    print("Результат умножения исправленного слова на матрицу G:", V2)

Исходное сообщение:  [1 0 0 0 1 1 0 0 0 1 1]
Слово с однократной ошибкой: [1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1]

Исправленное слово: [1 0 0 0 1 1 0 0 0 1 1]
Результат умножения исправленного слова на матрицу G: [1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1]


In [259]:
# Эксперимент для двукратной ошибки
Err = generate_word_with_n_mistakes(G, 2)
print("Слово с двукратной ошибкой:", Err)

Decoded_word = major_algorithm(Err, 2, 4, len(G))
if Decoded_word is None:
   print("\nНеобходима повторная отправка сообщения")
else:
   print("Исправленное слово:", Decoded_word)
   V2 = Decoded_word.dot(G) % 2
   print("Результат умножения исправленного слова на матрицу G:", V2)

Исходное сообщение:  [1 0 0 0 1 1 0 0 0 1 1]
Слово с двукратной ошибкой: [1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1]

Необходима повторная отправка сообщения
